In [42]:
import numpy as np
from gekko import GEKKO
from scipy.integrate import odeint
import matplotlib.pyplot as plt

m = GEKKO(remote=False)    # create GEKKO model

halfsat_const = m.Param(0.1, name='halfsat')
N0 = m.Param(1., name='n0')
inflow_rate = m.Param(0.1, name='inflow rate')
mortality_rate = m.Param(0.1, name='mortality rate')

N = m.SV(1, lb=0, name='nut')

P = m.Array(m.SV, 4)
for var in P:
    var.lower = 0
    var.value = 0.1

Parr = [m.SV(0.1, name='P'+str(i)) for i in range(4)]


In [43]:
def nuptake(N, P, halfsat):
    return N/(N+halfsat)*P

@np.vectorize
def vec_nuptake(N, P, halfsat):
    return N/(N+halfsat)*P

In [48]:
np.vectorize(nuptake)(N,Parr,halfsat_const)

array([((((nut)/((nut+halfsat))))*(p0)), ((((nut)/((nut+halfsat))))*(p1)),
       ((((nut)/((nut+halfsat))))*(p2)), ((((nut)/((nut+halfsat))))*(p3))],
      dtype=object)

In [59]:
testfunc = np.frompyfunc(nuptake, 3, 1)

testfunc(N,P,halfsat_const)

array([((((nut)/((nut+halfsat))))*(v2)), ((((nut)/((nut+halfsat))))*(v3)),
       ((((nut)/((nut+halfsat))))*(v4)), ((((nut)/((nut+halfsat))))*(v5))],
      dtype=object)

In [47]:
[m.Intermediate(var) for var in vec_nuptake(N,Parr,halfsat_const)]

[0, 0, 0, 0]

In [ ]:
t = np.arange(0,10,0.2)
m.time = t

# Growth under nutrient limitation is described via Monod / Michaelis-Menten kinetics
nutlim = m.Intermediate(N/(N+halfsat_const)*P)
N_influx = m.Intermediate(N0 * inflow_rate)
mortality = m.Intermediate(P * mortality_rate)

m.Equation(N.dt()==N_influx - nutlim)
m.Equations([[P[i].dt()==nutlim - mortality] for i in range(len(P))])

m.options.NODES = 3
m.options.IMODE = 7

m.solve(disp=True)

plt.plot(m.time, N, 'r--', label='N Gekko')
plt.plot(m.time, P, 'b--', label='P Gekko')
plt.legend()
plt.show()